![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Alpha Streams Portfolio Optimization Notebook
<br >

## Introduction
This notebook demonstrates how to optimize the allocation given to each alpha in a portfolio of alphas. We take a walk-forward approach where we rebalance the portfolio every month in an effort to maximize the portfolio's Sortino ratio. To avoid any look-ahead bias, we optimize the allocations given to the alphas using only their historical equity curves as we step through time.
  
<br >
    
## Specify User Settings
Here we declare which Alpha Streams algorithms we'd like to analyze. The `alpha_id_by_name` dictionary below includes the name of each alpha and its corresponding ID. The ID is retreived from the URL of the alpha listing in the [Alpha Market](https://www.quantconnect.com/market).

At the bottom of this cell, we select how long of a lookback period to use in the Sortino ratio calculation.

In [1]:
# Select alphas
alpha_id_by_name = {'Statistical Risk Premium Arbitrage': 'f50519a3d2b2598960bcdb59d',
                    'Trident Oscillator': '79aeef32e4249e48dbb8c9a4d',
                    'Analytical Reversal Pattern Recognition': '82619f53d30e7a9c9a0853ce2',
                    'Volvesting': '1551edaa94e8d3127bfd51174',
                    'Harry\'s Healthcare Emporium': 'e38e0cc523131ca2db722b884',
                    'GEdge - Long Stocks Long Term Reversal (no ETF)': '83e0e49b953617aa109174a70',
                    'Waikikamukau': 'c5a00d5f82157bb94136c581f'}

# Set Sortino lookback window length
lookback = 45 # days

## Gather Alpha Streams Equity Curves
Here we retreive the live equity curves of the alphas using [this CSV](https://s3.amazonaws.com/alphastreams.quantconnect.com/alphas/equity-unified-live-factors.csv) from the [Alpha Market](https://www.quantconnect.com/terminal/#alpha-market). Since we need multiple equity curves in order to perform optimization, the curves are truncated at the beginning so atleast 2 alphas are always running live. To ensure all the equity curves are on the same scale with a starting value of 1, the curves are normalized.

In [2]:
from fetch_data import get_live_equity_curves
equity_curves = get_live_equity_curves(alpha_id_by_name)
equity_curves.plot(figsize=(16, 6))
plt.show()
equity_curves

## Determine Alpha Allocations Over Time
Here we call the `optimize_allocations` method. The method steps through the `equity_curves` DataFrame. Each month, we select a lookback window on the live equity curve of each alpha. The optimizer tests a collection of different weights for the alphas across these trailing equity curves. The allocation weights that optimize the portfolio Sortino ratio are selected, saved into a time series, and returned from the method call. 

Each month, we aim to allocate 100% of the portfolio among the alphas. The allocation given to each alpha is between the 0 and 100%. Through the optimization, the weight given to each Alpha Streams algorithm varies over time. Below, we plot some charts and display a table to show the dynamic allocations.

In [3]:
from calculations import optimize_allocations
allocations_over_time = optimize_allocations(equity_curves, lookback)

from plotting import plot_allocations_over_time
plot_allocations_over_time(allocations_over_time)
allocations_over_time.applymap('{:,.1%}'.format)

The latest allocation decision is saved in the last row of the table above.

In [4]:
print('The current allocation is:')
allocations_over_time.applymap('{:,.1%}'.format).tail(1)

## Compute Equity Curve of Composite Alpha
Using the `equity_curves` and the `allocations_over_time` table, we can compute the optimized equity curve of the portfolio. This is the equity curve we would of had if we followed the `allocations_over_time` in real time.

In [5]:
from calculations import get_composite_equity_curve
composite_equity_curve = get_composite_equity_curve(equity_curves, allocations_over_time)
composite_equity_curve.plot(figsize=(16, 6))
plt.show()

## Gather Performance Statistics
We now calculate several performance statistics on the `composite_equity_curve` above.

In [6]:
from statistics import get_performance_statistics
get_performance_statistics(composite_equity_curve)

## Display All of the Alpha Equity Curves
Here we plot the live equity curves of all the Alpha Streams algorithms, along with the optimized portfolio equity curve.

In [7]:
from plotting import plot_all_equity_curves
plot_all_equity_curves(equity_curves, composite_equity_curve)